In [1]:
import matplotlib.pyplot as plt
import math as m
import numpy as np  # 1.7 or higher
from ipywidgets import interactive
from IPython.display import display



import qgrid
import pandas as pd

import sympy as sym
sym.init_printing(order="none")
from sympy.vector import CoordSys3D
import sympy.vector as symvec


np.set_printoptions(suppress=True, formatter={'float_kind':'{:14.4f}'.format})  
%matplotlib inline

#  jupyter nbconvert --no-input --to pdf RotorHead.ipynb 

### Introduction
 
 I wanted to learn more about how a helicopter works, specifically how the rotors are controlled.  So I built a rotorhead out of LEGO and did some math.  I also relearned a lot about typesetting equations I knew about a decade ago.  I also learned about a method of helicopter control known as cyclic-collective pitch mixing, which is the algorithm I'll be discssing here.
 
### Definitions

<dl>
    
<dt>Helicopter</dt>
<dd>An aircraft that uses airflow over a rotating wing to generate lift, rather than using thrust to push a fixed wing through the air as in a traditional airplane.  A generic helicopter has a main rotor that generates life, and a tail rotor that counteracts the yaw induced by the main rotor torque on the fusulage and provides direcional control.  The main rotor of is attached to the fusulage by an assembly known as the rotorhead.  The prop shaft for the main rotor emerges from the transmission through the top of the fusulage.</dd>
    
    
    
    
<dt>Second Term</dt>
<dd>This is one definition of the second term. </dd>
<dd>This is another definition of the second term.</dd>
</dl>

In [5]:
P0 = sym.Symbol(r'P_O');  #Origin of XYZ system
Rcoll = sym.Symbol(r'R_{coll}') #height of collective above origin
Rsw = sym.Symbol(r'R_{sw}') #radius of swashplate arms from P_coll

Ap = sym.Symbol(r'\angle{A_p}') #pitch angle
Ar = sym.Symbol(r'\angle{A_r}') #roll angle
Acyl = sym.Symbol(r'\angle{A_{cyl}}') # cylinder spacing


Cf, C0 = sym.symbols(r'\vec{C_f},\vec{C_0}') 
Cp, C1 = sym.symbols(r'\vec{C_p},\vec{C_1}')
Cs, C2 = sym.symbols(r'\vec{C_s},\vec{C_2}')

# Front linear actuator, vector representing length
Ff, F0 = sym.symbols(r'\vec{F_f},\vec{F_0}') 
Fp, F1 = sym.symbols(r'\vec{F_p},\vec{F_1}') #Port linear actuator
Fs, F2 = sym.symbols(r'\vec{F_s},\vec{F_2}') #Starboard linear actuator

Vmast = sym.symbols(r'\vec{V_{mast}}')
Vdisk = sym.symbols(r'\vec{V_{disk}}')

Visectn, Visectnhat = sym.symbols(r'\vec{V_{isectN}},\hat{V_{isectN}}')
Vcn, Cn, Fn= sym.symbols(r'\vec{V_{cN}},\vec{C_N},\vec{F_N}')
Van, Pfn = sym.symbols(r'\vec{V_{aN}}, P_{fn}')

Paf, Pa0 = sym.symbols(r'P_a_f,P_a_0') #End of front swashplate arm
Pap, Pa1 = sym.symbols(r'P_a_p,P_a_1') #End of port swashplate arm
Pas, Pa2 = sym.symbols(r'P_a_s,P_a_2') #End of starboard swashplate arm

Pff, Pf0 = sym.symbols(r'P_f_f,P_f_0') #End of front swashplate foot
Pfp, Pf1 = sym.symbols(r'P_f_p,P_f_1') #End of port swashplate foot
Pfs, Pf2 = sym.symbols(r'P_f_s,P_f_2') #End of starboard swashplate foot

# cylinder minimum length, cylinder maximum length
Cmin, Cmax = sym.symbols("C_m_i_n C_m_a_x")

#Pitch vector from cyclic, Roll vector from cyclic
Vp, Vr = sym.symbols(r'\vec{V_p} \vec{V_r}')
Vrhat = sym.Symbol(r'\hat{V_r}') #unit vector versons of above
Vphat = sym.Symbol(r'\hat{V_p}')

display((P0, Rcoll, Rsw, Ap, Ar, Cf, C0, Cp, C1, Cs, C2))
display((Paf, Pa0, Pap, Pa1, Pas, Pa2, Pff, Pf0, Pfn))
display((Pfp, Pf1, Pfs, Pf2, Cmin, Cmax, Vcn, Van, Cn, Fn))
display((Visectn, Visectnhat, Vmast, Vdisk,  Vp, Vr, Vrhat, Vphat))

In [3]:
def EqsP(A, expr1, expr2, comment=""):
    A.append( (sym.Eq(expr1, expr2, evaluate=False), comment) )
    
eqs1 = []
EqsP(eqs1, Vp, sym.Matrix([sym.cos(Ap), 0, sym.sin(Ap)]).T)
EqsP(eqs1, Vphat, Vp/abs(Vp))

EqsP(eqs1, Vr, sym.Matrix([0, sym.cos(Ar), sym.sin(Ar)]).T)
EqsP(eqs1, Vrhat, Vr/abs(Vr))

EqsP(eqs1, Ff, sym.Matrix([Rsw, 0, 0]).T)
EqsP(eqs1, Fp, sym.Matrix([Rsw*sym.sin(Acyl), Rsw*sym.cos(Acyl), 0]).T)
EqsP(eqs1, Fs, sym.Matrix([Rsw*sym.sin(2*Acyl), Rsw*sym.cos(2*Acyl), 0]).T)

EqsP(eqs1, Vmast, sym.Matrix([0, 0, Rcoll]).T)

# order is important here
EqsP(eqs1, Vdisk, symvec.Cross(Vphat, Vrhat))

# for each cylinder, Vcn normal of plane formed by foot and mast
eqs2 = []
EqsP(eqs2, Fn, Pfn-P0)
EqsP(eqs2, Vcn, symvec.Cross(Fn, Vmast))
EqsP(eqs2, Visectn, symvec.Cross(Vdisk, Vcn))
EqsP(eqs2, Visectnhat, Visectn/abs(Visectn))
EqsP(eqs2, Van, Rsw*Visectnhat+Vmast)
EqsP(eqs2, Cn, Van - Fn)

def showeqs(eqs, offset=1):
    for i, ecom in enumerate(eqs):
        numsym = sym.Symbol(f"{i+offset}")
        (e, com) = ecom
        display(f"comment: {com}")
        display((numsym, e))

showeqs(eqs1)
display("Then, for every Cylinder")
display(Cn)
showeqs(eqs2, len(eqs1)+1)



'comment: '

(1, \vec{V_p} = [cos(\angle{A_p})  0  sin(\angle{A_p})])

'comment: '

'comment: '

(3, \vec{V_r} = [0  cos(\angle{A_r})  sin(\angle{A_r})])

...

'comment: '

'comment: '

'comment: '

'comment: '

'comment: '

### Source Code of Solve Function

```python
def solve(self, pitch, roll, collpct):
    Vp = lin.vector(m.cos(m.radians(pitch)), 
                    0, 
                    m.sin(m.radians(pitch)))
    Vr = lin.vector(0, 
                    m.cos(m.radians(roll)), 
                    m.sin(m.radians(roll)))
    # whatever
    
```

In [18]:
def hbyte(arr, off):
    i = 256*int(arr[off+0],16) +16*int(arr[off+1],16) + int(arr[off+2],16)
    return i  

n = 3
[(16**(n-i-1), i) for i in range(n)]

arr = "123"
a = hbyte(arr, 0)

b = sum([(16**(n-i-1)*int(arr[0+i])) for i in range(n)])
a, b

### Algorithm

The point Pcoll is first chosen from the collective input.  The pitch vector Vp is the x axis rotated
around the y axis by the pitch input.  The roll vector Vr is the y axis rotated along the x axis by
the roll input.

The normal vector representing the plane of the rotor then Vr X Vx.

Then for each of the cylinder, the Mast X the vector from the orgin to the foot of the cylinder
is normal to the plane the cylinder can travel in.  The cross product of this vector and the rotor
plane vector is the vector of intersection of these two planes.  This vector is normalized and then
multiplied by radius R and added to Pc to locate the top point of the cylinder.

Finally, the length of the cylinder vector is checked to make sure it fits within the limits.